In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
from scipy.integrate import odeint
import paho.mqtt.client as mqtt #import the client1
import time

In [2]:
def f(y, t, params):
    theta, omega = y      # unpack current values of y
    Q, d, Omega = params  # unpack parameters
    derivs = [omega,      # list of dy/dt=f functions
             -omega/Q + np.sin(theta) + d*np.cos(Omega*t)]
    return derivs

def solver():
    # Parameters
    Q = 2.0          # quality factor (inverse damping)
    d = 1.5          # forcing amplitude
    Omega = 0.65     # drive frequency

    # Initial values
    theta0 = 0.0     # initial angular displacement
    omega0 = 0.0     # initial angular velocity

    # Bundle parameters for ODE solver
    params = [Q, d, Omega]

    # Bundle initial conditions for ODE solver
    y0 = [theta0, omega0]

    # Make time array for solution
    tStop = 200.
    tInc = 0.05
    t = np.arange(0., tStop, tInc)

    # Call the ODE solver
    #psoln = odeint(f, y0, t, args=(params,))
    psoln = []
    cnt = 0
    
    print(len(t))
    for x in range(len(t)-1):
        y = odeint(f, y0, [t[x], t[x+1]], args=(params,))
        y0 = y[1]
        np.append (psoln, y)
        cnt = cnt + 1
        
        if(cnt>= 5):
            client.publish("output",y[0,0])
            cnt = 0;
        
        for k in range(20000):
            a = k + 1

In [3]:
modelo_str = ''

def on_message(client, userdata, message):
    modelo_str = str(message.payload.decode("utf-8"))
    print('inicio')
    solver()
    print('fim')

broker_address="127.0.0.1"

client = mqtt.Client("P1") #create new instance

client.on_message=on_message #attach function to callback

client.connect(broker_address) #connect to broker

client.loop_start() #start the loop

client.subscribe("modelo/#")
#client.publish("modelo",2)
#time.sleep(20) # wait
#client.loop_stop() #stop the loop

(0, 1)

inicio
4000
fim
